In [2]:
from pathlib import Path
import mlflow

TSP_EXPERIMENTS = {
    603996644784169297: "hynky-TSP-Verbal",
    193975243763088023: "hynky-TSP-Culture",
    842945248455049410: "hynky-TSP-Critical",
    996016737840831842: "hynky-TSP-Analytical"
}

mlflow.set_tracking_uri("../mlruns")

TSP_runs = mlflow.search_runs(experiment_names=TSP_EXPERIMENTS.values())
klokan_runs = mlflow.search_runs(experiment_names=["hynky-klokan-qa-train"])

# TSP_runs['params.split'] = TSP_runs['params.dataset_split'].where(TSP_runs['params.dataset_split'].notnull(), TSP_runs['params.split'])

In [3]:
TSP_runs

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.accuracy,params.metrics,params.model_name,params.prompts,...,params.model_max_tokens,params.dataset_size,params.outputs,params.model_type,tags.mlflow.source.type,tags.mlflow.source.name,tags.mlflow.user,tags.mlflow.runName,tags.mlflow.source.git.commit,tags.mlflow.loggedArtifacts
0,189e155fa98f4769b3285ac25a2de327,603996644784169297,FINISHED,file:///Users/hynky/Projects/LLM-Eval/mlruns/6...,2024-03-16 22:39:15.293000+00:00,2024-03-16 22:47:04.688000+00:00,0.612613,"[{'name': 'accuracy', 'inputs': ['correct_answ...",anthropic/claude-3-sonnet,{'system_prompt_path': 'prompts/qa/system/cz_s...,...,2048,None,"[{'key': 'answer', 'transform': {'name': 'extr...",openrouter,LOCAL,/Users/hynky/Projects/LLM-Eval/czeval/run.py,hynky,hynky-TSP-Verbal-anthropic/claude-3-sonnet-Q&A...,f762d328b9575483962ec75da6b5da98a466ab66,"[{""path"": ""dataset.json"", ""type"": ""table""}]"
1,e12b53d696e94ab08c18d28c7295a27c,603996644784169297,FINISHED,file:///Users/hynky/Projects/LLM-Eval/mlruns/6...,2024-03-16 22:33:03.676000+00:00,2024-03-16 22:39:12.361000+00:00,0.627628,"[{'name': 'accuracy', 'inputs': ['correct_answ...",mistralai/mistral-large,{'system_prompt_path': 'prompts/qa/system/cz_s...,...,2048,None,"[{'key': 'answer', 'transform': {'name': 'extr...",openrouter,LOCAL,/Users/hynky/Projects/LLM-Eval/czeval/run.py,hynky,hynky-TSP-Verbal-mistralai/mistral-large-Q&A-2...,f762d328b9575483962ec75da6b5da98a466ab66,"[{""path"": ""dataset.json"", ""type"": ""table""}]"
2,58b0c092fdc44b999fe433e34f99e9b4,193975243763088023,FINISHED,file:///Users/hynky/Projects/LLM-Eval/mlruns/1...,2024-03-16 22:25:47.717000+00:00,2024-03-16 22:39:13.839000+00:00,0.834635,"[{'name': 'accuracy', 'inputs': ['correct_answ...",anthropic/claude-3-sonnet,{'system_prompt_path': 'prompts/qa/system/cz_s...,...,2048,None,"[{'key': 'answer', 'transform': {'name': 'extr...",openrouter,LOCAL,/Users/hynky/Projects/LLM-Eval/czeval/run.py,hynky,hynky-TSP-Culture-anthropic/claude-3-sonnet-Q&...,f762d328b9575483962ec75da6b5da98a466ab66,"[{""path"": ""dataset.json"", ""type"": ""table""}]"
3,8702ced25e9b4674894e01dca21365af,193975243763088023,FINISHED,file:///Users/hynky/Projects/LLM-Eval/mlruns/1...,2024-03-16 22:19:31.318000+00:00,2024-03-16 22:33:02.074000+00:00,0.859375,"[{'name': 'accuracy', 'inputs': ['correct_answ...",mistralai/mistral-large,{'system_prompt_path': 'prompts/qa/system/cz_s...,...,2048,None,"[{'key': 'answer', 'transform': {'name': 'extr...",openrouter,LOCAL,/Users/hynky/Projects/LLM-Eval/czeval/run.py,hynky,hynky-TSP-Culture-mistralai/mistral-large-Q&A-...,f762d328b9575483962ec75da6b5da98a466ab66,"[{""path"": ""dataset.json"", ""type"": ""table""}]"
4,141b39b7f9114aaab345fdffaa6b38ba,842945248455049410,FINISHED,file:///Users/hynky/Projects/LLM-Eval/mlruns/8...,2024-03-16 22:15:14.233000+00:00,2024-03-16 22:25:46.165000+00:00,0.688928,"[{'name': 'accuracy', 'inputs': ['correct_answ...",anthropic/claude-3-sonnet,{'system_prompt_path': 'prompts/qa/system/cz_s...,...,2048,None,"[{'key': 'answer', 'transform': {'name': 'extr...",openrouter,LOCAL,/Users/hynky/Projects/LLM-Eval/czeval/run.py,hynky,hynky-TSP-Critical-anthropic/claude-3-sonnet-Q...,f762d328b9575483962ec75da6b5da98a466ab66,"[{""path"": ""dataset.json"", ""type"": ""table""}]"
5,7266a53284cf43e9880d6a899ee1a8a0,842945248455049410,FINISHED,file:///Users/hynky/Projects/LLM-Eval/mlruns/8...,2024-03-16 22:09:28.007000+00:00,2024-03-16 22:19:30.030000+00:00,0.667838,"[{'name': 'accuracy', 'inputs': ['correct_answ...",mistralai/mistral-large,{'system_prompt_path': 'prompts/qa/system/cz_s...,...,2048,None,"[{'key': 'answer', 'transform': {'name': 'extr...",openrouter,LOCAL,/Users/hynky/Projects/LLM-Eval/czeval/run.py,hynky,hynky-TSP-Critical-mistralai/mistral-large-Q&A...,f762d328b9575483962ec75da6b5da98a466ab66,"[{""path"": ""dataset.json"", ""type"": ""table""}]"
6,f1225fa2214942348a6a7b7cfb04907a,603996644784169297,FINISHED,file:///Users/hynky/Pr

In [4]:
TSP_runs[TSP_runs["params.model_name"] == "anthropic/claude-2.1"]["params.split"]


KeyError: 'params.split'

In [15]:
TSP_runs["params.dataset_split"]

0         Verbal
1         Verbal
2        Culture
3        Culture
4       Critical
5       Critical
6         Verbal
7        Culture
8       Critical
9     Analytical
10        Verbal
11    Analytical
12       Culture
13    Analytical
14      Critical
15    Analytical
16        Verbal
17       Culture
18      Critical
19    Analytical
20        Verbal
21       Culture
22      Critical
23    Analytical
24        Verbal
25       Culture
26      Critical
27    Analytical
28        Verbal
29       Culture
30      Critical
31    Analytical
32        Verbal
33       Culture
34      Critical
35    Analytical
36        Verbal
37       Culture
38      Critical
39    Analytical
40        Verbal
41       Culture
42      Critical
43    Analytical
44       Culture
45        Verbal
46      Critical
47    Analytical
Name: params.dataset_split, dtype: object

In [17]:
## General table
res = TSP_runs[["metrics.accuracy", "params.dataset_split", "params.model_name"]].pivot_table(index="params.model_name", columns="params.dataset_split", values="metrics.accuracy")

# Rename the params.split to task
res = res.rename(columns={"Analytical": "analytical", "Critical": "critical", "Culture": "culture", "Verbal": "verbal"})

# Join with Klokan
klokan = klokan_runs[["metrics.accuracy", "params.model_name"]].pivot_table(index="params.model_name", values="metrics.accuracy")
klokan.rename(columns={"metrics.accuracy": "klokan"}, inplace=True)

joined = res.join(klokan)
# Rename index params.models_name to model_name
joined.index.rename('model_name', inplace=True)



In [18]:
joined

,analytical,critical,culture,verbal,klokan
model_name,,,,,
anthropic/claude-2.1,0.380403,0.644991,0.798177,0.633634,0.382388
anthropic/claude-3-haiku,0.332373,0.604569,0.816406,0.624625,0.322263
anthropic/claude-3-opus,0.472622,0.764499,0.924479,0.801802,0.578106
anthropic/claude-3-sonnet,0.378482,0.688928,0.834635,0.612613,0.442804
cohere/command-r,0.278578,0.534271,0.704427,0.444444,0.248462
google/gemini-pro,0.289145,0.609842,0.807292,0.609610,0.286593
google/gemma-7b-it,0.221902,0.279438,0.229167,0.228228,0.169742
mistralai/mistral-large,0.385207,0.667838,0.859375,0.627628,0.410824
mistralai/mistral-medium,0.312200,0.595782,0.773438,0.504505,0.293973


In [19]:

import os

# Create the directory if it does not exist
if not os.path.exists("leaderboard"):
    os.makedirs("leaderboard")


joined.to_csv("leaderboard/table.csv")

In [43]:

import pandas as pd

# Klokan spider preparation

# Function to process each artifact and calculate accuracy
def process_artifact(artifact_info):
    artifact_info = artifact_info.replace("file:", "")
    try:
        dataset_path = Path(artifact_info)
        if (dataset_path / "dataset.json").exists():
            with open(dataset_path / "dataset.json", "r") as f:
                x = pd.read_json(f, lines=True)
        elif (dataset_path / "dataset.csv").exists():
            x = pd.read_json(dataset_path / "dataset.csv", lines=True)
        else:
            print(f"Neither dataset.json nor dataset.csv could be found in {dataset_path}")
            raise FileNotFoundError("Neither dataset.json nor dataset.csv could be found.")
    except:
        return None
    # Parse into a real DataFrame
    real_df = pd.DataFrame.from_records(x["data"][0], columns=x["columns"][0])

    # Add a column indicating whether the answer was correct
    real_df["is_correct"] = real_df.apply(lambda row: row["answer"] == row["correct_answer"], axis=1)

    # Group by 'category' and calculate mean accuracy
    category_accuracy = real_df.groupby("category")["is_correct"].mean()

    return category_accuracy

# Process each artifact and store the results
all_category_accuracies = [(process_artifact(artifact[1]["artifact_uri"]), artifact[1].get("params.model_name")) for artifact in klokan_runs.iterrows()]
combined_accuracies = pd.concat([x[0].rename(x[1]) for x in all_category_accuracies], axis=1).T
df = combined_accuracies
df = df* 100

# Rename the columns
categories = [
    "Elementary 2-3",
    "Elementary 4-5",
    "Elementary 6-7",
    "Elementary 8-9",
    "High School 1-2",
    "High School 3-4"
]

df.columns = categories
df.sort_index(inplace=True)

df.to_csv("leaderboard/klokan.csv")


In [41]:
!pip install plotly nbformat

  Using cached fastjsonschema-2.19.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached jsonschema-4.21.1-py3-none-any.whl.metadata (7.8 kB)
  Using cached jsonschema_specifications-2023.12.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached rpds_py-0.18.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (4.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 1.8 MB/s eta 0:00:00a 0:00:01
Using cached jsonschema-4.21.1-py3-none-any.whl (85 kB)
Using cached fastjsonschema-2.19.1-py3-none-any.whl (23 kB)
Using cached jsonschema_specifications-2023.12.1-py3-none-any.whl (18 kB)
Using cached rpds_py-0.18.0-cp312-cp312-macosx_11_0_arm64.whl (332 kB)


In [44]:
import pandas as pd
import plotly.graph_objects as go
import pandas as pd

import matplotlib.pyplot as plt

# Combine all category accuracies into a single DataFrame


categories = df.columns.tolist()
categories = [*categories, categories[0]]  # Ensure the graph is circular by appending the start to the end
colors = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf',   # blue-teal
    '#f7b6d2',  # pastel pink
    '#bcbd22',  # faded green
    '#dbdb8d',  # light olive
    '#17becf',  # soft blue
    '#9edae5',  # light blue
    '#c5b0d5',  # soft purple
    '#c49c94',  # dusty rose
    '#f7b6d2',  # pastel pink
    '#bcbd22',  # faded green
    '#dbdb8d',  # light olive
    '#17becf',  # soft blue
    '#9edae5',  # light blue
    '#c5b0d5',  # soft purple
    '#c49c94',  # dusty rose
]

# Setting for 1000x1000
fig_1000 = go.Figure()

for i, (idx, row) in enumerate(df.iterrows()):
    row = row.tolist() + [row[0]]  # Ensure the graph is circular by appending the start to the end
    color = colors[i]
    fig_1000.add_trace(go.Scatterpolar(
        r=row,
        theta=categories,
        opacity=0.4,
        name=idx,
        line=dict(color=color, width=4),  # Adjust line width for better visibility
    ))

fig_1000.update_layout(
    width=600,
    height=950,
    polar=dict(
        angularaxis=dict(
            gridwidth=2,  # Increase line width for better visibility
            rotation=90,
        direction='clockwise',

        ),
        radialaxis=dict(

            visible=True,
            range=[0, 100],
            angle=45,
            tickangle=45,
            tickvals=[0, 25, 50, 75, 100],
            ticktext=["0%", "25%", "50%", "75%", "100%"],
        ),
    ),
    title_text='Klokan-QA - Accuracy',
    title_x=0.5,
    title_y=0.97,
    title_xanchor='center',
    title_yanchor='top',
    title_font_size=24,
    title_font_color='#333333',
    font=dict(
        family='Arial',
        size=16,
        color='#333333'
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.45,
        xanchor="center",
        x=0.5
    )
)






/var/folders/cj/cyw80lln0mz_9hlly0x3qvf80000gn/T/ipykernel_29817/2967686727.py:43: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'line': {'color': '#1f77b4', 'width': 4},
              'name': 'anthropic/claude-2.1',
              'opacity': 0.4,
              'r': [43.96551724137931, 50.35971223021583, 39.87730061349693,
                    39.75155279503105, 33.33333333333333, 14.772727272727273,
                    43.96551724137931],
              'theta': [Elementary 2-3, Elementary 4-5, Elementary 6-7, Elementary
                        8-9, High School 1-2, High School 3-4, Elementary 2-3],
              'type': 'scatterpolar'},
             {'line': {'color': '#ff7f0e', 'width': 4},
              'name': 'anthropic/claude-3-haiku',
              'opacity': 0.4,
              'r': [37.5, 38.405797101449274, 27.160493827160494, 31.25,
                    32.70440251572327, 25.609756097560975, 37.5],
              'theta': [Elementary 2-3, Elementary 4-5, Elementary 6-7, Elementary
                        8-9, High School 1-2, High School 3-4, Elementary 2-3],
              'type': 'scatterpolar'},
             {'line': {'color': '#2ca02c', 'width': 4},
              'name': 'anthropic/claude-3-opus',
              'opacity': 0.4,
              'r': [78.57142857142857, 65.94202898550725, 61.111111111111114,
                    50.0, 45.28301886792453, 48.78048780487805, 78.57142857142857],
              'theta': [Elementary 2-3, Elementary 4-5, Elementary 6-7, Elementary
                        8-9, High School 1-2, High School 3-4, Elementary 2-3],
              'type': 'scatterpolar'},
             {'line': {'color': '#d62728', 'width': 4},
              'name': 'anthropic/claude-3-sonnet',
              'opacity': 0.4,
              'r': [59.82142857142857, 60.86956521739131, 38.2716049382716, 45.0,
                    33.9622641509434, 25.609756097560975, 59.82142857142857],
              'theta': [Elementary 2-3, Elementary 4-5, Elementary 6-7, Elementary
                        8-9, High School 1-2, High School 3-4, Elementary 2-3],
              'type': 'scatterpolar'},
             {'line': {'color': '#9467bd', 'width': 4},
              'name': 'cohere/command-r',
              'opacity': 0.4,
              'r': [24.107142857142858, 28.26086956521739, 29.01234567901235,
                    21.875, 24.528301886792452, 18.29268292682927,
                    24.107142857142858],
              'theta': [Elementary 2-3, Elementary 4-5, Elementary 6-7, Elementary
                        8-9, High School 1-2, High School 3-4, Elementary 2-3],
              'type': 'scatterpolar'},
             {'line': {'color': '#8c564b', 'width': 4},
              'name': 'google/gemini-pro',
              'opacity': 0.4,
              'r': [33.92857142857143, 28.985507246376812, 26.543209876543212,
                    28.125, 30.18867924528302, 23.170731707317074,
                    33.92857142857143],
              'theta': [Elementary 2-3, Elementary 4-5, Elementary 6-7, Elementary
                        8-9, High School 1-2, High School 3-4, Elementary 2-3],
              'type': 'scatterpolar'},
             {'line': {'color': '#e377c2', 'width': 4},
              'name': 'google/gemma-7b-it',
              'opacity': 0.4,
              'r': [16.964285714285715, 15.942028985507244, 16.666666666666664,
                    12.5, 20.125786163522015, 21.951219512195124,
                    16.964285714285715],
              'theta': [Elementary 2-3, Elementary 4-5, Elementary 6-7, Elementary
                        8-9, High School 1-2, High School 3-4, Elementary 2-3],
              'type': 'scatterpolar'},
             {'line': {'color': '#7f7f7f', 'width': 4},
              'name': 'mistralai/mistral-large',
              'opacity': 0.4,
              'r': [51.78571428571429, 53.62318840579711, 41.358024691358025,
                    37.5, 35.22012578616352, 23.170731707317074, 51.78571428571429],
              'theta': [Elementary 2-3, Elementary 4-5, Elementary 6-7, Elementary
                        8-9, High School 1-2, High Sch

In [27]:
import numpy as np
df = pd.pivot_table(TSP_runs, values='metrics.accuracy', index=['params.model_name'], columns=['params.dataset_split'], aggfunc=np.mean)
# Convert the values to percentages
df = df * 100

# Rename splits Culture -> Cultural overview
# Analytical -> Analytical thinking
# Verbal -> Verbal thinking
# Critical -> Critical thinking

df.rename(columns={
    'Culture': 'Cultural',
}, inplace=True)

# Sort the rows by alpah
# Sort the DataFrame by model name in alphabetical order
df.sort_index(inplace=True)
df.index.name = ""
df.to_csv("leaderboard/tsp.csv")


/var/folders/cj/cyw80lln0mz_9hlly0x3qvf80000gn/T/ipykernel_29817/1679758744.py:2: FutureWarning:

The provided callable <function mean at 0x113346660> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.



In [38]:
import plotly.graph_objects as go
import pandas as pd


categories = df.columns.tolist()
categories = [*categories, categories[0]]  # Ensure the graph is circular by appending the start to the end
colors = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf',   # blue-teal
    '#f7b6d2',  # pastel pink
    '#bcbd22',  # faded green
    '#dbdb8d',  # light olive
    '#17becf',  # soft blue
    '#9edae5',  # light blue
    '#c5b0d5',  # soft purple
    '#c49c94',  # dusty rose
    '#f7b6d2',  # pastel pink
    '#bcbd22',  # faded green
    '#dbdb8d',  # light olive
    '#17becf',  # soft blue
    '#9edae5',  # light blue
    '#c5b0d5',  # soft purple
    '#c49c94',  # dusty rose
]

# Setting for 1000x1000
fig_1000 = go.Figure()

for i, (idx, row) in enumerate(df.iterrows()):
    row = row.tolist() + [row[0]]  # Ensure the graph is circular by appending the start to the end
    color = colors[i]
    fig_1000.add_trace(go.Scatterpolar(
        r=row,
        theta=categories,
        opacity=0.4,
        name=idx,
        line=dict(color=color, width=4),  # Adjust line width for better visibility
    ))

fig_1000.update_layout(
    width=600,
    height=950,
    polar=dict(
        angularaxis=dict(
            gridwidth=2,  # Increase line width for better visibility
        ),
        radialaxis=dict(
            visible=True,
            range=[0, 100],
            angle=45,
            tickangle=45,
            tickvals=[0, 25, 50, 75, 100],
            ticktext=["0%", "25%", "50%", "75%", "100%"],
        ),
    ),
    title_text='TSP-QA - Accuracy',
    title_x=0.5,
    title_y=0.97,
    title_xanchor='center',
    title_yanchor='top',
    title_font_size=24,
    title_font_color='#333333',
    font=dict(
        family='Arial',
        size=16,
        color='#333333'
    ),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.45,
        xanchor="center",
        x=0.5
    )
)



/var/folders/cj/cyw80lln0mz_9hlly0x3qvf80000gn/T/ipykernel_29817/413848649.py:38: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'line': {'color': '#1f77b4', 'width': 4},
              'name': 'anthropic/claude-2.1',
              'opacity': 0.4,
              'r': [38.04034582132565, 64.49912126537785, 79.81770833333334,
                    63.36336336336337, 38.04034582132565],
              'theta': [Analytical, Critical, Cultural, Verbal, Analytical],
              'type': 'scatterpolar'},
             {'line': {'color': '#ff7f0e', 'width': 4},
              'name': 'anthropic/claude-3-haiku',
              'opacity': 0.4,
              'r': [33.23727185398655, 60.45694200351493, 81.640625,
                    62.46246246246246, 33.23727185398655],
              'theta': [Analytical, Critical, Cultural, Verbal, Analytical],
              'type': 'scatterpolar'},
             {'line': {'color': '#2ca02c', 'width': 4},
              'name': 'anthropic/claude-3-opus',
              'opacity': 0.4,
              'r': [47.262247838616716, 76.44991212653778, 92.44791666666666,
                    80.18018018018019, 47.262247838616716],
              'theta': [Analytical, Critical, Cultural, Verbal, Analytical],
              'type': 'scatterpolar'},
             {'line': {'color': '#d62728', 'width': 4},
              'name': 'anthropic/claude-3-sonnet',
              'opacity': 0.4,
              'r': [37.848222862632085, 68.89279437609842, 83.46354166666666,
                    61.261261261261254, 37.848222862632085],
              'theta': [Analytical, Critical, Cultural, Verbal, Analytical],
              'type': 'scatterpolar'},
             {'line': {'color': '#9467bd', 'width': 4},
              'name': 'cohere/command-r',
              'opacity': 0.4,
              'r': [27.857829010566764, 53.427065026362044, 70.44270833333334,
                    44.44444444444444, 27.857829010566764],
              'theta': [Analytical, Critical, Cultural, Verbal, Analytical],
              'type': 'scatterpolar'},
             {'line': {'color': '#8c564b', 'width': 4},
              'name': 'google/gemini-pro',
              'opacity': 0.4,
              'r': [28.914505283381363, 60.98418277680141, 80.72916666666666,
                    60.96096096096096, 28.914505283381363],
              'theta': [Analytical, Critical, Cultural, Verbal, Analytical],
              'type': 'scatterpolar'},
             {'line': {'color': '#e377c2', 'width': 4},
              'name': 'google/gemma-7b-it',
              'opacity': 0.4,
              'r': [22.19020172910663, 27.943760984182774, 22.916666666666664,
                    22.822822822822822, 22.19020172910663],
              'theta': [Analytical, Critical, Cultural, Verbal, Analytical],
              'type': 'scatterpolar'},
             {'line': {'color': '#7f7f7f', 'width': 4},
              'name': 'mistralai/mistral-large',
              'opacity': 0.4,
              'r': [38.52065321805956, 66.78383128295255, 85.9375,
                    62.76276276276276, 38.52065321805956],
              'theta': [Analytical, Critical, Cultural, Verbal, Analytical],
              'type': 'scatterpolar'},
             {'line': {'color': '#bcbd22', 'width': 4},
              'name': 'mistralai/mistral-medium',
              'opacity': 0.4,
              'r': [31.21998078770413, 59.57820738137083, 77.34375,
                    50.45045045045045, 31.21998078770413],
              'theta': [Analytical, Critical, Cultural, Verbal, Analytical],
              'type': 'scatterpolar'},
             {'line': {'color': '#17becf', 'width': 4},
              'name': 'mistralai/mixtral-8x7b-instruct',
              'opacity': 0.4,
              'r': [25.26416906820365, 51.14235500878734, 71.22395833333334,
                    43.54354354354354, 25.26416906820365],
              'theta': [Analytical, Critical, Cultural, Verbal, Analytical],
              'type': 'scatterpolar'},
             {'line': {'color': '#f7b6d2', 'width': 4},
              'name': 'openai/gpt-3.5-turbo',
              'opacity': 0.4,
            

In [380]:
from datasets import  load_dataset

x = load_dataset("hynky/TSP")

In [446]:
q_names = []
vals = []
for dst in x.values():
    lst = [
        x+y for x,y in zip(dst["questionName"], dst["file"])
    ]
    q_names.extend(lst)
    vals.extend(dst)



In [450]:
import numpy as np

# get non uniques ones

from collections import Counter
name_counts = Counter(q_names)
non_unique_names = [name for name, count in name_counts.items() if count > 1]
print(non_unique_names)

# Get indexes of non unique names
non_unique_indexes = [i for i, name in enumerate(q_names) if name in non_unique_names]
print(non_unique_indexes)


['TSP 2010 - varianta 03, otázka č. 4 / 70./Users/hynekkdylicek/Projects/MUNI-TSP/downloads/tsp2010_v03.qdefx', 'TSP 2010 - varianta 03, otázka č. 2 / 70./Users/hynekkdylicek/Projects/MUNI-TSP/downloads/tsp2010_v03.qdefx', 'TSP 2010 - varianta 03, otázka č. 3 / 70./Users/hynekkdylicek/Projects/MUNI-TSP/downloads/tsp2010_v03.qdefx']
[1381, 1399, 1435, 1489, 1533, 1558]


In [469]:
vals[1399]

{'question': 'Vyberte nesprávné tvrzení.',
 'questionTopic': 'Kritické myšlení',
 'questionName': 'TSP 2010 - varianta 03, otázka č. 2 / 70.',
 'file': '/Users/hynekkdylicek/Projects/MUNI-TSP/downloads/tsp2010_v03.qdefx',
 'correct_answer': 'A',
 'answers.A': 'Z ustanovení vyplývá, že před výpovědí nájmu bez přivolení soudu musí pronajímatel doručit nájemci písemnou výstrahu.',
 'answers.B': 'Z ustanovení vyplývá, že za okolností stanovených Občanským zákoníkem má nájemce při výpovědi z nájmu nárok na náhradní bydlení.',
 'answers.C': 'Ustanovení připouští, že nájemce může dostat výpověď z nájmu bez přivolení soudu také v případě, že dluží platby za užívání bytu za leden, červenec a září.',
 'answers.D': 'Ustanovení připouští, že občané mohou bez vážných důvodů užívat dva a více bytů.',
 'answers.E': 'Ustanovení připouští, že výpověď z nájmu lze podat i z jiných než uvedených důvodů.',
 '__index_level_0__': 1869}

In [468]:
vals[1558]

{'question': 'Zpráva z Internetu o podpoře v nezaměstnanosti: „Do 50 let náleží nezaměstnanému podpora 6 měsíců, od 50 do 55 let pak 9 měsíců a starší 55 let mohou pobírat podporu až 12 měsíců. V prvních dvou měsících je vyplácena podpora 80 % z průměrného čistého výdělku, od třetího měsíce pak 55 % z průměrného čistého výdělku. Podpora v nezaměstnanosti má strop 13 307 Kč měsíčně.“ Vyberte správné tvrzení.',
 'questionTopic': 'Kritické myšlení',
 'questionName': 'TSP 2010 - varianta 03, otázka č. 2 / 70.',
 'file': '/Users/hynekkdylicek/Projects/MUNI-TSP/downloads/tsp2010_v03.qdefx',
 'correct_answer': 'D',
 'answers.A': 'Maximální celková částka podpory vyplacená během nezaměstnanosti je pro všechny nezaměstnané stejná.',
 'answers.B': 'Čím je věk občana nižší, tím nižší celková částka podpory mu bude v případě nezaměstnanosti vyplacena.',
 'answers.C': 'Z ustanovení nelze činit závěry, neboť z něj není patrné, co přesně se rozumí průměrným čistým výdělkem.',
 'answers.D': 'Dva občan